In [2]:
import numpy as np
import random as rd
# Constants
GENOME_SIZE = 8 # N-QUEENS (EX: 'every Queen has a genome size of 8')
POPULATION_SIZE = 100
NUM_OFFSPRING = 2
RECOMBINATION_RATE = 1.0  
MUTATION_RATE = 0.8  
MAX_FITNESS_EVALUATIONS = 10000

In [3]:
# random init of individuals in population of size POPULATION_SIZE
def init_population(GENOME_SIZE, POPULATION_SIZE):
    # low while is included, however, high value is excluded in the calcs
    return np.random.randint(low=1, high=8+1, size=(POPULATION_SIZE, GENOME_SIZE))

In [4]:
population = init_population(GENOME_SIZE, POPULATION_SIZE)
population.shape

(100, 8)

In [199]:
def conflict_counter(individual):
    # To Check: 
    # 1] Queens can't be in the same row (due to individual representation, no need to check for conflicting columns)
    # 2] Queens can't be in the same diagonal
    # Fitness Functions returns: '#(Conflicting Pairs of Queens)'
    n = len(individual)
    conflict_counter = 0
    for Q1 in range(n):
        for Q2 in range(Q1+1, n):
            # counts conflicts row-wise
            if(individual[Q1] == individual[Q2]):
                conflict_counter += 1
            # counts conflicts diag-wise
            # EX: [_,1,_,_,_,_,6,_] -> (2,1) & (7,6) => DIAG: (2-7)=(1-6) <=> (-5)=(-5)
            # DIFF_X = DIFF_Y => DIAG 
            if abs(individual[Q1] - individual[Q2]) == abs(Q1 - Q2):
                conflict_counter += 1
    return conflict_counter

# returns an fitness array of current population (may be 1 individual in population or more) 
def eval_fitness(population):
    fitness_evals = []
    if(np.array(population).ndim == 1):
            fitness_evals.append(1 - conflict_counter(population) / 28) 
    elif(np.array(population).ndim == 2):
        for i in range(len(population)):
            fitness_evals.append(1 - conflict_counter(population[i]) / 28) 
    return fitness_evals

In [200]:
print("Initial Max-Fitness Evaluation", max(eval_fitness(population)))

Initial Max-Fitness Evaluation 0.8928571428571429


In [201]:
# Select the best 2 selected_parents from a random group of 5 individuals
def parent_selection(population):
    candidates = rd.sample(list(population), 5)
    candidates.sort(key=eval_fitness, reverse=True)
    return candidates[0], candidates[1]

In [202]:
selected_parents = parent_selection(population)
selected_parents

(array([8, 5, 4, 8, 1, 7, 7, 8]), array([8, 2, 6, 3, 2, 7, 6, 3]))

In [205]:
# at crossover point (cut_index), cutt both parents DNA and crossfill
def cut_and_crossfill(dad, mom, GENOME_SIZE):
    child_one = []
    child_two = []
    # cut_index = np.random.randint(1, GENOME_SIZE-1)
    cut_index = GENOME_SIZE/2
    for i in range(GENOME_SIZE):
        if(i < cut_index):
            child_one.append(dad[i])
            child_two.append(mom[i])
        else:
            child_one.append(mom[i])
            child_two.append(dad[i])
    return [child_one, child_two]

# with prob recombination_rate, do cut_and_crossfill
def recombination(selected_parents, recombination_rate):
    if (np.random.rand() <= recombination_rate):
        dad = selected_parents[0]
        mom = selected_parents[1]
        children = cut_and_crossfill(dad, mom, GENOME_SIZE)
        return children
    else:
        return selected_parents


In [206]:
selected_parents = parent_selection(population)
offspring = recombination(selected_parents, RECOMBINATION_RATE)
print(selected_parents, np.mean(eval_fitness(selected_parents)))
print(offspring, np.mean(eval_fitness(offspring)))

(array([5, 2, 2, 6, 8, 2, 5, 1]), array([2, 7, 3, 4, 1, 5, 5, 8])) 0.8392857142857143
[[5, 2, 2, 6, 1, 5, 5, 8], [2, 7, 3, 4, 8, 2, 5, 1]] 0.8035714285714286


In [207]:
def swap_mutation(individual, mutation_rate, GENOME_SIZE):
    if (np.random.rand() <= mutation_rate):
        # get indices of 2 random genes in the genome
        gene_idx1, gene_idx2 = np.random.choice(GENOME_SIZE, 2, replace=False)

        # return the genes at specified indices
        gene1, gene2 = individual[gene_idx1].copy(), individual[gene_idx2].copy()

        # swap the genes contents of the two genes at the specified indices
        individual[gene_idx1] = gene2
        individual[gene_idx2] = gene1
    return individual

In [208]:
print(offspring, np.mean(eval_fitness(offspring)))
mutated_offspring = [swap_mutation(child, MUTATION_RATE, GENOME_SIZE) for child in offspring]
print(mutated_offspring, np.mean(eval_fitness(mutated_offspring)))

[[5, 2, 2, 6, 1, 5, 5, 8], [2, 7, 3, 4, 8, 2, 5, 1]] 0.8035714285714286
[[5, 2, 2, 6, 5, 1, 5, 8], [2, 7, 3, 4, 8, 2, 5, 1]] 0.7857142857142858


In [210]:
offspring_fitness = eval_fitness(mutated_offspring)

[0.7142857142857143, 0.8571428571428572]

In [239]:
def survival_selection(population, offspring):
    population_list = population.tolist()
    population_list.sort(key=eval_fitness, reverse=False)        
    population_list[0], population_list[1] = offspring[0], offspring[1]
    return np.array(population_list)

In [240]:
population_list = population.tolist()
population_list.sort(key=eval_fitness, reverse=False)
print(eval_fitness(population_list[0]))
print(eval_fitness(population_list[88]))
print(eval_fitness(population_list[-1]))

[0.5]
[0.8214285714285714]
[0.8928571428571429]


In [241]:
print(np.mean(eval_fitness(population)))
new_population = survival_selection(population, offspring)
print(np.mean(eval_fitness(new_population)))

0.7246428571428573
0.73


In [ ]:
# def genetic_algorithm():
#     CURR_FITNESS_EVALUATIONS  = 0
#     population = init_population(GENOME_SIZE, POPULATION_SIZE)

#     while( not(termination_condition(CURR_FITNESS_EVALUATIONS, MAX_FITNESS_EVALUATIONS)) ):
#         selected_parents = parent_selection(population)
#         offspring = recombination(selected_parents, RECOMBINATION_RATE)
#         mutated_offspring = [swap_mutation(child, MUTATION_RATE, GENOME_SIZE) for child in offspring]
#         offspring_fitness = eval_fitness(mutated_offspring)
#         CURR_FITNESS_EVALUATIONS += len(offspring_fitness)
#         population = survival_selection(population, mutated_offspring)